# TensorFlow Low-Level API

高レベル API を使わない、いわゆる生の TensorFlow でコードを書いてみましょう。

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

print(tf.__version__)

TensorFlow 付属のモジュールを使って MNIST データセットをダウンロードします。

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

学習用のデータを流し込むための tf.placeholder を作成します。

In [ ]:
x_ph = tf.placeholder(tf.float32, [None, 784])
y_ph = tf.placeholder(tf.float32, [None, 10])

ニューラルネットの weight を tf.Variable として作成します。
行列積の計算なども自分で記述する形になります。

In [ ]:
weights = tf.Variable(tf.random_normal([784, 20], stddev=0.1))
biases = tf.Variable(tf.zeros([20]))
hidden = tf.matmul(x_ph, weights) + biases

weights = tf.Variable(tf.random_normal([20, 10], stddev=0.1))
biases = tf.Variable(tf.zeros([10]))
logits = tf.matmul(hidden, weights) + biases

y = tf.nn.softmax(logits)

ここから先は tf.layers を使ったときと全く同じです。

In [ ]:
cross_entropy = -tf.reduce_mean(y_ph * tf.log(y))

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_ph, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
train_op = tf.train.GradientDescentOptimizer(1e-1).minimize(cross_entropy)

In [ ]:
init_op = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(10001):
        x_train, y_train = mnist.train.next_batch(100)
        sess.run(train_op, feed_dict={x_ph: x_train, y_ph: y_train})
        if i % 100 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={x_ph: x_train, y_ph: y_train})
            test_loss = sess.run(cross_entropy, feed_dict={x_ph: mnist.test.images, y_ph: mnist.test.labels})
            tf.logging.info("Iteration: {0} Training Loss: {1} Test Loss: {2}".format(i, train_loss, test_loss))
    test_accuracy = sess.run(accuracy, feed_dict={x_ph: mnist.test.images, y_ph: mnist.test.labels})
    tf.logging.info("Accuracy: {}".format(test_accuracy))